In [9]:
import pandas as pd
import numpy as np
from tqdm import tqdm

#读取数据
df = pd.read_csv(r'C:\Users\86130\Desktop\论文\日本会议发表\data\app_use_info_label.csv', sep=',', header=0)
df2 = pd.read_csv(r'C:\Users\86130\Desktop\论文\日本会议发表\data\user_portrait.csv', sep=',', header=0)
df3 = pd.read_csv(r'C:\Users\86130\Desktop\论文\日本会议发表\data\user_trajectory2.csv', sep=',', header=0).drop(columns='Unnamed: 0')
#合并数据
df_all = pd.merge(df, df2, left_on='msisdn', right_on='userid', how='left').drop(columns = ['userid'])
df_all = df_all.merge(df3, on = 'msisdn', how = 'left')

In [10]:
#日期特征处理
def get_time_feature(df, col):
    
    df_copy = df.copy()
    prefix = col + "_"
    df_copy['new_'+col] = df_copy[col].astype(str)
    
    col = 'new_'+col
    df_copy[col] = pd.to_datetime(df_copy[col])
    df_copy[prefix + 'year'] = df_copy[col].dt.year
    df_copy[prefix + 'month'] = df_copy[col].dt.month
    df_copy[prefix + 'day'] = df_copy[col].dt.day
    df_copy[prefix + 'dayofweek'] = df_copy[col].dt.dayofweek
    df_copy[prefix + 'is_wknd'] = df_copy[col].dt.dayofweek // 6
    df_copy[prefix + 'quarter'] = df_copy[col].dt.quarter
    df_copy[prefix + 'is_month_start'] = df_copy[col].dt.is_month_start.astype(int)
    df_copy[prefix + 'is_month_end'] = df_copy[col].dt.is_month_end.astype(int)
    del df_copy[col]
    
    return df_copy   
    
df_all = get_time_feature(df_all, 'stime')
df_all = get_time_feature(df_all, 'end_time')

In [11]:
#label编码
from sklearn import preprocessing
 
enc=preprocessing.LabelEncoder() 
enc=enc.fit(df_all['app_class_1']) 
df_all['app_class_1']=enc.transform(df_all['app_class_1'])

enc2=preprocessing.LabelEncoder() 
enc2=enc2.fit(df_all['app_class_2']) 
df_all['app_class_2']=enc2.transform(df_all['app_class_2'])

In [12]:
#特征衍生
df_all['up_flow']= df_all['up_flow']/1024
df_all['down_flow']= df_all['down_flow']/1024

df_all['app1_upflow_mean'] = df_all['app_class_1'].map(df_all.groupby('app_class_1')['up_flow'].mean())
df_all['app2_upflow_mean'] = df_all['app_class_2'].map(df_all.groupby('app_class_2')['up_flow'].mean())
df_all['app1_upflow_max'] = df_all['app_class_1'].map(df_all.groupby('app_class_1')['up_flow'].max())
df_all['app2_upflow_max'] = df_all['app_class_2'].map(df_all.groupby('app_class_2')['up_flow'].max())
df_all['app1_upflow_min'] = df_all['app_class_1'].map(df_all.groupby('app_class_1')['up_flow'].min())
df_all['app2_upflow_min'] = df_all['app_class_2'].map(df_all.groupby('app_class_2')['up_flow'].min())

df_all['app1_downflow_mean'] = df_all['app_class_1'].map(df_all.groupby('app_class_1')['down_flow'].mean())
df_all['app2_downflow_mean'] = df_all['app_class_2'].map(df_all.groupby('app_class_2')['down_flow'].mean())
df_all['app1_downflow_max'] = df_all['app_class_1'].map(df_all.groupby('app_class_1')['down_flow'].max())
df_all['app2_downflow_max'] = df_all['app_class_2'].map(df_all.groupby('app_class_2')['down_flow'].max())
df_all['app1_downflow_min'] = df_all['app_class_1'].map(df_all.groupby('app_class_1')['down_flow'].min())
df_all['app2_downflow_min'] = df_all['app_class_2'].map(df_all.groupby('app_class_2')['down_flow'].min())

df_all['app1_consume_mean'] = df_all['app_class_1'].map(df_all.groupby('app_class_1')['consume'].mean())
df_all['app2_consume_mean'] = df_all['app_class_2'].map(df_all.groupby('app_class_2')['consume'].mean())
df_all['app1_consume_max'] = df_all['app_class_1'].map(df_all.groupby('app_class_1')['consume'].max())
df_all['app2_consume_max'] = df_all['app_class_2'].map(df_all.groupby('app_class_2')['consume'].max())
df_all['app1_consume_min'] = df_all['app_class_1'].map(df_all.groupby('app_class_1')['consume'].min())
df_all['app2_consume_min'] = df_all['app_class_2'].map(df_all.groupby('app_class_2')['consume'].min())

df_all['app1_age_mean'] = df_all['app_class_1'].map(df_all.groupby('app_class_1')['age'].mean())
df_all['app2_age_mean'] = df_all['app_class_2'].map(df_all.groupby('app_class_2')['age'].mean())
df_all['app1_age_max'] = df_all['app_class_1'].map(df_all.groupby('app_class_1')['age'].max())
df_all['app2_age_max'] = df_all['app_class_2'].map(df_all.groupby('app_class_2')['age'].max())
df_all['app1_age_min'] = df_all['app_class_1'].map(df_all.groupby('app_class_1')['age'].min())
df_all['app2_age_min'] = df_all['app_class_2'].map(df_all.groupby('app_class_2')['age'].min())

df_all['up-down'] = df_all['up_flow']-df_all['down_flow']
df_all['up/down'] = df_all['up_flow']/df_all['down_flow']
df_all['up+down'] = df_all['up_flow']+df_all['down_flow']

In [13]:
#分离训练集，测试集
from sklearn.model_selection import train_test_split

df_all=df_all.replace([np.inf, -np.inf], 0)
df_all.fillna(0,inplace=True)
train_df,test_df = train_test_split(df_all,train_size=0.8,shuffle=True,random_state=42)
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)
feature_cols = [cols for cols in df_all if cols not in ['msisdn','label','end_time','stime','times_month']]
len(feature_cols)

63

In [14]:
#构建模型
import optuna
import xgboost as xgb
import lightgbm as lgb
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier 
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.metrics import roc_auc_score,accuracy_score,classification_report
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from pytorch_tabnet import tab_model
from sklearn.svm import SVC
from pytorch_tabnet.multitask import TabNetMultiTaskClassifier
import torch
import warnings
warnings.filterwarnings('ignore')

# # 标准化
# def scale(train_features,test_features):
#     scaler=StandardScaler()
#     scaler.fit(train_features)
#     train_features=pd.DataFrame(scaler.transform(train_features),columns=test_features.keys())
#     test_features=pd.DataFrame(scaler.transform(test_features),columns=test_features.keys())
#     return train_features,test_features

# 模型定义
def ml_model(clf,train_x,train_y,test_x=[],test_y=[]):
    seeds=[888]
    train_oof = np.zeros([train_x.shape[0],3])
    feat_imp_df = pd.DataFrame()
    feat_imp_df['feature'] = train_x.columns
    feat_imp_df['imp'] = 0
    #标准化
    scaler=StandardScaler()
    scaler.fit(train_x)
    train_x = scaler.transform(train_x)
    if (len(test_x)!=0) and (len(test_x)!=0):
        test_oof = np.zeros([test_x.shape[0],3])
        test_x = scaler.transform(test_x)
    for seed in seeds:
        print('Seed:',seed)
        folds = 5
        kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)
        acc_scores_val = []
        acc_scores_test = []
        for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
            trn_x, trn_y, val_x, val_y = train_x[train_index], train_y[train_index], train_x[valid_index], \
                                        train_y[valid_index] 
            if clf == 'xgb':
                print("|  XGB  Fold  {}  Training Start           |".format(str(i + 1)))
                xgb_params = {'booster': 'gbtree','objective': 'multi:softprob','eval_metric':'mlogloss','num_class':3,
                    'n_estimators':500,'max_depth': 8,'lambda': 10,'subsample': 0.7,'colsample_bytree': 0.8,'eta': 0.1,
                    'colsample_bylevel': 0.7,'tree_method': 'hist','seed': seed,'nthread': 16}
                #训练模型
                model = xgb.XGBClassifier(*xgb_params)
                model.fit(trn_x,trn_y,eval_set=[(trn_x, trn_y),(val_x,val_y)],early_stopping_rounds=50,verbose=100)
                #验证集
                print('************ Val_Result ************')
                val_pred  = model.predict_proba(val_x)
                acc_score_val = accuracy_score(val_y, np.argmax(val_pred, axis=1))
                acc_scores_val.append(acc_score_val)
                print('AVG_acc :',sum(acc_scores_val)/len(acc_scores_val))
                print('XGB_result :',classification_report(val_y, np.argmax(val_pred, axis=1)))
                #测试集
                if (len(test_x)!=0) and (len(test_x)!=0):
                    print('************ Test_Result ************')
                    test_pred  = model.predict_proba(test_x)
                    acc_score_tset = accuracy_score(test_y, np.argmax(test_pred, axis=1))
                    acc_scores_test.append(acc_score_tset)
                    print('AVG_acc :',sum(acc_scores_test)/len(acc_scores_test))
                    print('XGB_result :',classification_report(test_y, np.argmax(test_pred, axis=1)))
                    #保存测试集结果
                    test_oof += test_pred / kf.n_splits / len(seeds)
                
                #保存训练集结果
                train_oof[valid_index] = val_pred / kf.n_splits / len(seeds)
                
                #模型特征重要性
                feat_imp_df['imp'] += model.feature_importances_ / folds/ len(seeds)
                feat_imp_df = feat_imp_df.sort_values(by='imp', ascending=False).reset_index(drop=True)
                feat_imp_df['rank'] = range(feat_imp_df.shape[0])
            if clf == 'lgb':
                lgb_params = {'boosting_type': 'gbdt','n_estimators':500,'min_child_weight': 4,'num_leaves': 64,
                    'feature_fraction': 0.8,'bagging_fraction': 0.8,'bagging_freq': 4,'learning_rate': 0.02,
                    'seed': seed,'nthread': 32,'n_jobs':8,'verbose': -1}
                print("|  LGB  Fold  {}  Training Start           |".format(str(i + 1)))
                #训练模型
                model = lgb.LGBMClassifier(**lgb_params)
                model.fit(trn_x,trn_y)
                
                #验证集
                print('************ Val_Result ************')
                val_pred  = model.predict_proba(val_x)
                acc_score_val = accuracy_score(val_y, np.argmax(val_pred, axis=1))
                acc_scores_val.append(acc_score_val)
                print('AVG_acc :',sum(acc_scores_val)/len(acc_scores_val))
                print('Cat_result :',classification_report(val_y, np.argmax(val_pred, axis=1)))
                #测试集
                if (len(test_x)!=0) and (len(test_x)!=0):
                    print('************ Test_Result ************')
                    test_pred  = model.predict_proba(test_x)
                    acc_score_tset = accuracy_score(test_y, np.argmax(test_pred, axis=1))
                    acc_scores_test.append(acc_score_tset)
                    print('AVG_acc :',sum(acc_scores_test)/len(acc_scores_test))
                    print('Cat_result :',classification_report(test_y, np.argmax(test_pred, axis=1))) 
                    #保存测试集结果
                    test_oof += test_pred / kf.n_splits / len(seeds)
                #保存训练集结果
                train_oof[valid_index] = val_pred / kf.n_splits / len(seeds)
            if clf == 'cat':
                print("|  cat  Fold  {}  Training Start           |".format(str(i + 1)))
                #训练模型
                model = CatBoostClassifier(verbose=False)
                model.fit(trn_x,trn_y)
                    
                #验证集
                print('************ Val_Result ************')
                val_pred  = model.predict_proba(val_x)
                acc_score_val = accuracy_score(val_y, np.argmax(val_pred, axis=1))
                acc_scores_val.append(acc_score_val)
                print('AVG_acc :',sum(acc_scores_val)/len(acc_scores_val))
                print('Cat_result :',classification_report(val_y, np.argmax(val_pred, axis=1)))
                #测试集
                if (len(test_x)!=0) and (len(test_x)!=0):
                    print('************ Test_Result ************')
                    test_pred  = model.predict_proba(test_x)
                    acc_score_tset = accuracy_score(test_y, np.argmax(test_pred, axis=1))
                    acc_scores_test.append(acc_score_tset)
                    print('AVG_acc :',sum(acc_scores_test)/len(acc_scores_test))
                    print('Cat_result :',classification_report(test_y, np.argmax(test_pred, axis=1))) 
                    #保存测试集结果
                    test_oof += test_pred / kf.n_splits / len(seeds)
                    
                #保存结果
                train_oof[valid_index] = val_pred / kf.n_splits / len(seeds)
            if clf == 'svm':
                print("|  SVM  Fold  {}  Training Start           |".format(str(i + 1)))
                #训练模型
                model = SVC(kernel='rbf', C=1, gamma='auto', probability=True,max_iter=1000)
                model.fit(trn_x,trn_y)
                
                #验证集
                print('************ Val_Result ************')
                val_pred  = model.predict_proba(val_x)
                acc_score_val = accuracy_score(val_y, np.argmax(val_pred, axis=1))
                acc_scores_val.append(acc_score_val)
                print('AVG_acc :',sum(acc_scores_val)/len(acc_scores_val))
                print('Svm_result :',classification_report(val_y, np.argmax(val_pred, axis=1)))
                #测试集
                print('************ Test_Result ************')
                test_pred  = model.predict_proba(test_x)
                acc_score_tset = accuracy_score(test_y, np.argmax(test_pred, axis=1))
                acc_scores_test.append(acc_score_tset)
                print('AVG_acc :',sum(acc_scores_test)/len(acc_scores_test))
                print('Svm_result :',classification_report(test_y, np.argmax(test_pred, axis=1))) 
                #保存结果
                train_oof[valid_index] = val_pred / kf.n_splits / len(seeds)
                test_oof += test_pred / kf.n_splits / len(seeds)
                
            if clf == 'tabnet':
                print(f"     Tab_model  Fold {i+1}  Training Starting       ")
                if torch.cuda.is_available():
                    print("Using GPU")
                    device = "cuda"
                else:
                    print("Using CPU")
                    device = "cpu"
                    
                torch.manual_seed(seed)
                np.random.seed(seed)
                model = tab_model.TabNetClassifier()

                model.fit(trn_x, trn_y,eval_set=[(val_x, val_y)],eval_metric=['accuracy'])
                #验证集
                print('************ Val_Result ************')
                val_pred  = model.predict_proba(val_x)
                acc_score_val = accuracy_score(val_y, np.argmax(val_pred, axis=1))
                acc_scores_val.append(acc_score_val)
                print('AVG_acc :',sum(acc_scores_val)/len(acc_scores_val))
                print('Tabnet_result :',classification_report(val_y, np.argmax(val_pred, axis=1)))
                #测试集
                print('************ Test_Result ************')
                test_pred  = model.predict_proba(test_x)
                acc_score_tset = accuracy_score(test_y, np.argmax(test_pred, axis=1))
                acc_scores_test.append(acc_score_tset)
                print('AVG_acc :',sum(acc_scores_test)/len(acc_scores_test))
                print('Tabnet_result :',classification_report(test_y, np.argmax(test_pred, axis=1))) 
                #保存结果
                train_oof[valid_index] = val_pred / kf.n_splits / len(seeds)
                test_oof += test_pred / kf.n_splits / len(seeds)
        if (len(test_x)!=0) and (len(test_x)!=0):
            return train_oof,test_oof,model,scaler
        else:
            return train_oof,model,scaler

# 训练 XGB模型
xgb_train_oof_1,xgb_test_oof_1, xgb_model_1,scaler_1 = ml_model('xgb',train_df[feature_cols], train_df['label'],test_df[feature_cols], test_df['label'])

# 训练 LGB模型
lgb_train_oof_1,lgb_test_oof_1,lgb_model_1,scaler_1 = ml_model('lgb',train_df[feature_cols], train_df['label'],test_df[feature_cols], test_df['label'])

# 训练 CAT模型
cat_train_oof_1,cat_test_oof_1,cat_model_1,scaler_1 = ml_model('cat',train_df[feature_cols], train_df['label'],test_df[feature_cols], test_df['label'])

# # 训练 SVM模型
# svm_train_oof,svm_test_oof,svm_model = ml_model('svm',train_df[feature_cols], train_df['label'],test_df[feature_cols], test_df['label'])

# # 训练 Tabnet模型
# tab_train_oof,tab_test_oof,tab_model = ml_model('tabnet',train_df[feature_cols], train_df['label'],test_df[feature_cols], test_df['label'])

Seed: 888
|  XGB  Fold  1  Training Start           |
[0]	validation_0-mlogloss:0.89551	validation_1-mlogloss:0.89819
[99]	validation_0-mlogloss:0.31382	validation_1-mlogloss:0.44397
************ Val_Result ************
AVG_acc : 0.8014725934006
XGB_result :               precision    recall  f1-score   support

           0       0.88      0.89      0.89      5910
           1       0.70      0.73      0.72      3685
           2       0.71      0.62      0.66      1406

    accuracy                           0.80     11001
   macro avg       0.77      0.75      0.75     11001
weighted avg       0.80      0.80      0.80     11001

************ Test_Result ************
AVG_acc : 0.8061231910406516
XGB_result :               precision    recall  f1-score   support

           0       0.89      0.89      0.89      7458
           1       0.70      0.74      0.72      4501
           2       0.73      0.63      0.68      1792

    accuracy                           0.81     13751
   macro

In [15]:
#训练集困难样本处理
df_pre = pd.DataFrame()
df_pre['xgb_pre'] = np.argmax(xgb_train_oof_1,axis=1)
df_pre['lgb_pre'] = np.argmax(lgb_train_oof_1,axis=1)
df_pre['cat_pre'] = np.argmax(cat_train_oof_1,axis=1)
df_pre['label'] = train_df['label']

grade_list = []
for row in df_pre.itertuples():
    grade = 0
    if getattr(row,'xgb_pre') == getattr(row,'label'):
        grade += 1
    if getattr(row,'lgb_pre') == getattr(row,'label'):
        grade += 1
    if getattr(row,'cat_pre') == getattr(row,'label'):
        grade += 1
    grade_list.append(grade)

#困难样本识别
df_pre['grade'] = grade_list
train_hard_index = df_pre.loc[(df_pre['grade']==0)].index

train_hard_df = train_df.loc[train_hard_index].reset_index(drop=True)
xgb_train_hard_oof, xgb_train_hard_model,scaler_2 = ml_model('xgb',train_hard_df[feature_cols],train_hard_df['label'])
lgb_train_hard_oof, lgb_train_hard_model,scaler_2 = ml_model('lgb',train_hard_df[feature_cols],train_hard_df['label'])
cat_train_hard_oof, cat_train_hard_model,scaler_2 = ml_model('cat',train_hard_df[feature_cols],train_hard_df['label'])

#训练集替换困难样本结果
xgb_train_oof_2 = np.argmax(xgb_train_oof_1,axis=1)
xgb_train_oof_2[train_hard_index]=np.argmax(xgb_train_hard_oof,axis=1)

lgb_train_oof_2 = np.argmax(lgb_train_oof_1,axis=1)
lgb_train_oof_2[train_hard_index]=np.argmax(lgb_train_hard_oof,axis=1)

cat_train_oof_2 = np.argmax(cat_train_oof_1,axis=1)
cat_train_oof_2[train_hard_index]=np.argmax(cat_train_hard_oof,axis=1)

Seed: 888
|  XGB  Fold  1  Training Start           |
[0]	validation_0-mlogloss:0.96886	validation_1-mlogloss:0.98020
[99]	validation_0-mlogloss:0.20563	validation_1-mlogloss:0.62165
************ Val_Result ************
AVG_acc : 0.7434613244296049
XGB_result :               precision    recall  f1-score   support

           0       0.69      0.67      0.68       546
           1       0.84      0.85      0.85       771
           2       0.65      0.65      0.65       480

    accuracy                           0.74      1797
   macro avg       0.72      0.72      0.72      1797
weighted avg       0.74      0.74      0.74      1797

|  XGB  Fold  2  Training Start           |
[0]	validation_0-mlogloss:0.97101	validation_1-mlogloss:0.99008
[99]	validation_0-mlogloss:0.18730	validation_1-mlogloss:0.62019
************ Val_Result ************
AVG_acc : 0.736505286588759
XGB_result :               precision    recall  f1-score   support

           0       0.66      0.66      0.66       5

In [16]:
#测试集困难样本处理
df_pre = pd.DataFrame()
df_pre['xgb_pre'] = np.argmax(xgb_test_oof_1,axis=1)
df_pre['lgb_pre'] = np.argmax(lgb_test_oof_1,axis=1)
df_pre['cat_pre'] = np.argmax(cat_test_oof_1,axis=1).flatten()
df_pre['label'] = test_df['label']

grade_list = []
for row in df_pre.itertuples():
    grade = 0
    if getattr(row,'xgb_pre') == getattr(row,'label'):
        grade += 1
    if getattr(row,'lgb_pre') == getattr(row,'label'):
        grade += 1
    if getattr(row,'cat_pre') == getattr(row,'label'):
        grade += 1
    grade_list.append(grade)

#测试集替换困难样本结果
df_pre['grade'] = grade_list
test_hard_index = df_pre.loc[(df_pre['grade']==0)].index
test_hard_df = test_df.loc[test_hard_index].reset_index(drop=True)

print('Test_hard result :')
xgb_test_hard_oof = xgb_train_hard_model.predict(scaler_2.transform(test_hard_df[feature_cols]))
print('xgb_test_hard result:',accuracy_score(xgb_test_hard_oof,test_hard_df['label']))
lgb_test_hard_oof = lgb_train_hard_model.predict(scaler_2.transform(test_hard_df[feature_cols]))
print('lgb_test_hard result:',accuracy_score(lgb_test_hard_oof,test_hard_df['label']))
cat_test_hard_oof = cat_train_hard_model.predict(scaler_2.transform(test_hard_df[feature_cols]))
print('cat_test_hard result:',accuracy_score(cat_test_hard_oof,test_hard_df['label']))


xgb_test_oof_2 = np.argmax(xgb_test_oof_1,axis=1)
xgb_test_oof_2[test_hard_index]=xgb_test_hard_oof

lgb_test_oof_2 = np.argmax(lgb_test_oof_1,axis=1)
lgb_test_oof_2[test_hard_index]=lgb_test_hard_oof

cat_test_oof_2 = np.argmax(cat_test_oof_1,axis=1)
cat_test_oof_2[test_hard_index]=cat_test_hard_oof.flatten()

Test_hard result :
xgb_test_hard result: 0.7623337623337624
lgb_test_hard result: 0.7807807807807807
cat_test_hard result: 0.7734877734877735


In [ ]:
# def hard_sample(xgb_oof,lgb_oof,cat_oof,df):
#     df_pre = pd.DataFrame()
#     df_pre['xgb_pre'] = np.argmax(xgb_oof,axis=1)
#     df_pre['lgb_pre'] = np.argmax(lgb_oof,axis=1)
#     df_pre['cat_pre'] = np.argmax(cat_oof,axis=1)
#     df_pre['label'] = df['label']

#     grade_list = []
#     for row in df_pre.itertuples():
#         grade = 0
#         if getattr(row,'xgb_pre') == getattr(row,'label'):
#             grade += 1
#         if getattr(row,'lgb_pre') == getattr(row,'label'):
#             grade += 1
#         if getattr(row,'cat_pre') == getattr(row,'label'):
#             grade += 1
#         grade_list.append(grade)

#     #困难样本
#     df_pre['grade'] = grade_list
#     hard_index = df_pre.loc[(df_pre['grade']==0)].index

#     # 困难样本独立训练
#     # 训练 XGB模型
#     hard_df = df.loc[hard_index].reset_index(drop=True)
#     xgb_oof_2, xgb_model_2 = ml_model('xgb',hard_df[feature_cols],hard_df['label'])
#     # 训练 LGB模型
#     lgb_oof_2,lgb_model_2 = ml_model('lgb',hard_df[feature_cols], hard_df['label'])

#     # 训练 cat模型
#     cat_oof_2,cat_model_2 = ml_model('cat',hard_df[feature_cols], hard_df['label'])

#     #替换困难样本结果
#     xgb_oof = np.argmax(xgb_oof,axis=1)
#     xgb_oof[hard_index]=np.argmax(xgb_oof_2,axis=1)

#     lgb_oof = np.argmax(lgb_oof,axis=1)
#     lgb_oof[hard_index]=np.argmax(lgb_oof_2,axis=1)

#     cat_oof = np.argmax(cat_oof,axis=1)
#     cat_oof[hard_index]=np.argmax(cat_oof_2,axis=1)
    
#     return xgb_oof,lgb_oof,cat_oof

# xgb_train_oof,lgb_train_oof,cat_train_oof = hard_sample(xgb_train_oof,lgb_train_oof,cat_train_oof,train_df)
# xgb_test_oof,lgb_test_oof,cat_test_oof = hard_sample(xgb_test_oof,lgb_test_oof,cat_test_oof,test_df)

In [17]:

#xgb预测结果作为新特征(替换困难样本)
train_df['xgb_pre'] = xgb_train_oof_2
test_df['xgb_pre'] = xgb_test_oof_2
#lgb预测结果作为新特征(替换困难样本)
train_df['lgb_pre'] = lgb_train_oof_2
test_df['lgb_pre'] = lgb_test_oof_2
#cat预测结果作为新特征(替换困难样本)
train_df['cat_pre'] = cat_train_oof_2
test_df['cat_pre'] = cat_test_oof_2

#训练特征
feature_cols = [cols for cols in train_df if cols not in ['msisdn','label','end_time','stime','times_month']]
# 训练tabnet模型
final_tab_train_oof,final_tab_test_oof,final_tab_model,scaler = ml_model('tabnet',train_df[feature_cols], train_df['label'],test_df[feature_cols], test_df['label'])

Seed: 888
     Tab_model  Fold 1  Training Starting       
Using CPU
epoch 0  | loss: 0.53504 | val_0_accuracy: 0.92083 |  0:00:02s
epoch 1  | loss: 0.27773 | val_0_accuracy: 0.92155 |  0:00:04s
epoch 2  | loss: 0.25551 | val_0_accuracy: 0.92473 |  0:00:06s
epoch 3  | loss: 0.25235 | val_0_accuracy: 0.92592 |  0:00:08s
epoch 4  | loss: 0.25103 | val_0_accuracy: 0.92337 |  0:00:10s
epoch 5  | loss: 0.25265 | val_0_accuracy: 0.92446 |  0:00:12s
epoch 6  | loss: 0.24759 | val_0_accuracy: 0.92292 |  0:00:14s
epoch 7  | loss: 0.24627 | val_0_accuracy: 0.92782 |  0:00:16s
epoch 8  | loss: 0.24313 | val_0_accuracy: 0.92519 |  0:00:18s
epoch 9  | loss: 0.24407 | val_0_accuracy: 0.92273 |  0:00:20s
epoch 10 | loss: 0.2427  | val_0_accuracy: 0.92573 |  0:00:22s
epoch 11 | loss: 0.24061 | val_0_accuracy: 0.92437 |  0:00:24s
epoch 12 | loss: 0.24533 | val_0_accuracy: 0.92473 |  0:00:26s
epoch 13 | loss: 0.24438 | val_0_accuracy: 0.92573 |  0:00:28s
epoch 14 | loss: 0.24475 | val_0_accuracy: 0.9234

In [39]:
train_df[[col for col in feature_cols if col not in ['xgb_pre','lgb_pre','cat_pre']]]

,app_class_1,app_class_2,up_flow,down_flow,age,sex,consume,user_star,region,prov,...,app1_age_mean,app2_age_mean,app1_age_max,app2_age_max,app1_age_min,app2_age_min,up-down,up/down,up+down,tab_pre
0,31,336,427.747070,243.935547,40,0,100.20,3,1000250.0,32.0,...,36.728163,36.562149,75,74,-1,-1,183.811523,1.753525,671.682617,1
1,62,482,0.816406,3.180664,36,0,229.00,5,1000250.0,32.0,...,39.948128,39.910922,85,85,-1,-1,-2.364258,0.256678,3.997070,0
2,90,770,68.468750,3584.277344,52,0,88.19,2,0.0,0.0,...,35.645078,35.374341,74,74,-1,-1,-3515.808594,0.019103,3652.746094,1
3,119,739,846.987305,22409.771484,42,0,187.00,5,1000250.0,32.0,...,40.124583,40.116589,85,85,-1,-1,-21562.784180,0.037795,23256.758789,2
4,61,598,13.763672,183.606445,32,1,154.00,4,1000250.0,32.0,...,39.846303,36.451613,85,60,-1,21,-169.842773,0.074963,197.370117,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54999,62,482,30.463867,71.309570,32,0,110.20,4,1000523.0,32.0,...,39.948128,39.910922,85,85,-1,-1,-40.845703,0.427206,101.773438,1
55000,62,482,1.765625,5.877930,38,0,68.00,4,0.0,0.0,...,39.948128,39.910922,85,85,-1,-1,-4.112305,0.300382,7.643555,0
55001,8,719,2.119141,1.094727,51,0,88.99,4,0.0,0.0,...,40.120906,41.536608,85,85,-1,-1,1.024414,1.935772,3.213867,1
55002,62,421,114.936523,37.320312,56,0,8.00,1,0.0,0.0,...,39.948128,40.616967,85,85,-1,-1,77.616211,3.079731,152.256836,1


In [37]:
test_df[[col for col in feature_cols if col not in ['xgb_pre','lgb_pre','cat_pre']]]

,app_class_1,app_class_2,up_flow,down_flow,age,sex,consume,user_star,region,prov,...,app1_age_mean,app2_age_mean,app1_age_max,app2_age_max,app1_age_min,app2_age_min,up-down,up/down,up+down,tab_pre
0,3,221,0.691406,2.771484,60,0,22.56,1,0.0,0.0,...,37.497268,37.908537,76,76,16,16,-2.080078,0.249471,3.462891,0
1,8,719,6.082031,1.983398,58,1,91.10,2,1000250.0,32.0,...,40.120906,41.536608,85,85,-1,-1,4.098633,3.066470,8.065430,0
2,31,697,419.723633,543.608398,29,0,48.00,1,0.0,0.0,...,36.728163,36.865104,75,75,-1,-1,-123.884766,0.772107,963.332031,0
3,137,821,0.599609,0.577148,27,0,68.10,2,0.0,0.0,...,37.181319,38.666667,74,49,16,27,0.022461,1.038917,1.176758,0
4,31,697,364.347656,2061.189453,34,0,93.10,2,1000250.0,32.0,...,36.728163,36.865104,75,75,-1,-1,-1696.841797,0.176766,2425.537109,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13746,119,431,262.013672,207.897461,34,0,176.70,3,1000250.0,32.0,...,40.124583,44.354667,85,85,-1,17,54.116211,1.260302,469.911133,2
13747,139,714,1.764648,4.765625,34,1,191.23,4,1000250.0,32.0,...,35.606897,34.975207,75,75,17,17,-3.000977,0.370287,6.530273,0
13748,119,213,1.821289,1.342773,42,0,263.60,6,0.0,0.0,...,40.124583,36.940822,85,85,-1,-1,0.478516,1.356364,3.164062,0
13749,61,617,99.733398,55.399414,32,1,39.10,1,0.0,0.0,...,39.846303,39.991061,85,85,-1,-1,44.333984,1.800261,155.132812,1


In [ ]:
import joblib
#模型保存
def save_model(mdoel,save_path):
    # save_path = save_path + '/' +  mdoel_name +'.pkl'
    joblib.dump(mdoel, save_path)
#模型保存路径
xgb_model_path = './model/xgb.pkl'
lgb_model_path = './model/lgb.pkl'
cat_model_path = './model/cat.pkl'
svm_model_path = './model/svm.pkl'
tabnet_model_path = './model/tabnet.pkl'
final_tab_model_path = './model/final_tab.pkl'
#保存
save_model(xgb_model,xgb_model_path)
save_model(lgb_model, lgb_model_path)
save_model(cat_model,  cat_model_path)
save_model(svm_model,svm_model_path)
save_model(tab_model, tabnet_model_path)
save_model(final_tab_model,  final_tab_model_path)